In [1]:
{-# LANGUAGE OverloadedStrings #-}
{-# LANGUAGE DeriveGeneric #-}

import Data.ByteString.Lazy (readFile, writeFile)
import Control.Lens hiding ((.=))
import Data.Default.Class
import Data.Aeson
import Data.Aeson.Encode.Pretty
import Data.Aeson.Lens
import Data.Monoid ((<>))
import Data.Text (Text, pack, unpack)
import GHC.Generics
import Network.HTTP.Req
import Prelude hiding (readFile, writeFile)
import System.Process

In [2]:
data Project = Project
    { owner  :: Text
    , repo   :: Text
    , rev    :: Text
    , sha256 :: Text
    } deriving (Show, Generic)

instance FromJSON Project
instance ToJSON Project

In [3]:
extractProject :: Value -> Text -> Maybe Project
extractProject versions name = do
    prj <- versions ^? key name
    case fromJSON prj of
        Error _ -> Nothing
        Success p -> Just p

In [4]:
r :: (MonadHttp m, FromJSON a) => Project -> Text -> m (JsonResponse a)
r project branch = req GET
    (  https "api.github.com"
    /: "repos"
    /: owner project
    /: repo project
    /: "branches"
    /: branch
    ) NoReqBody jsonResponse (header "User-Agent" "vaibhavsagar")

In [5]:
getRev :: Project -> Text -> IO (Maybe Text)
getRev project branch = do
    res <- responseBody <$> runReq def (r project branch) :: IO Value
    return $ res ^? key "commit" . key "sha" . _String

In [6]:
buildURL :: Project -> Text
buildURL project
    =  "https://github.com/"
    <> owner project <> "/"
    <> repo project  <> "/"
    <> "archive"     <> "/"
    <> rev project   <> ".tar.gz" 

In [7]:
getSha256 :: Text -> Bool -> IO Text
getSha256 url doUnpack = let
    option = if doUnpack then ["--unpack"] else []
    in pack . init <$> readProcess "nix-prefetch-url" (option ++ [unpack url]) ""

In [8]:
update :: FilePath -> Text -> Text -> Bool -> IO ()
update filename projectName branchName doUnpack = do
    Just versions <- decode <$> readFile filename :: IO (Maybe Value)
    let (Just project) = extractProject versions projectName
    Just latestRev <- getRev project branchName
    latestSha256 <- getSha256 (buildURL project { rev = latestRev }) doUnpack
    let project' = project { rev = latestRev, sha256 = latestSha256 }
    let versions' = versions & key projectName .~ toJSON project'
    writeFile filename (encodePretty' defConfig { confIndent = Spaces 2, confCompare = compare } versions')

In [9]:
update "versions.json" "nixpkgs" "nixos-18.03" False